Implementing RFE on new dataset provided by Michelle. This notebook will include everything on the pipeline for day 0 including all the ML models on test data set and their performance evalation from John Hopkins. We decided that we will only do Random Forest for the first stage (or feature selection). 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import openpyxl
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.metrics import accuracy_score


from pandas import Series
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV

C:\Users\rajnishk\AppData\Local\Temp\ipykernel_15060\4126161524.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Dataset Preparation

In [2]:
df_CBC_umich = pd.read_excel(r'C:\Users\rajnishk\Dropbox (University of Michigan)\2023-09-25 CBC CART for Benjie and Rajnish from Mary\data folder 4-4-2024\cleaned_data_from_Rashmi\Umich_data_with_demography.xlsx')
df_CBC_JH = pd.read_excel(r'C:\Users\rajnishk\Dropbox (University of Michigan)\2023-09-25 CBC CART for Benjie and Rajnish from Mary\data folder 4-4-2024\Test_data_JHU\04-11-2024 JHMI data with changed names from JHMI Compiled CBC, No ALL, CRP_LOD_0.xlsx')

In [3]:
# We will have to keep same columns in both Umich and JH dataset. 
# print("Columns in Umich dataset are %s" %df_CBC_umich.columns)
# print("Columns in JH Test dataset are %s" %df_CBC_JH.columns)

Previously we had considered following columns for analysis: Index(['StudyID', 'WBC', 'NEU#', 'LYMP#', 'MON#', 'CRS_Grade', 'NT_Incidence',
       'HGB', 'HCT', 'PLT', 'RBC', 'MCV', 'MCH', 'MCHC', 'RDW', 'MPV', 'Sex',
       'Age', 'CRP', 'Ferritin'    dtype='objec. I will keep these same variables in the new dataset and drop all the other variables.t')

In [4]:
# This is temporary and I will come back to it. 
columns_for_analysis = ['studyid', 'dpi', 'wbc_count', 'hgb', 'hct', 'plt', 'rbc_count', 'mcv',
       'mch', 'mchc', 'rdw', 'mpv', 'absolute_neutrophil',
       'absolute_lymphocyte', 'absolute_monocyte',   'crp', 'ferritin', 
        'clinician_defined_crs_grade',  
        'NT_incidence', 
        'Age', 'Sex']

In [5]:
df_umich = df_CBC_umich[columns_for_analysis] # This is the data frame that will go into analysis.
df_JH = df_CBC_JH[columns_for_analysis] # This is the relevant columns from data set from John Hopkins


In [6]:
# df_umich

In [7]:
# df_umich.wbc_count[5]

In [8]:
# object_cols = df_umich.select_dtypes(include='object')
for col in df_umich.select_dtypes(include='object'):
    # print (col)
    df_umich[col] = df_umich[col].str.strip()

C:\Users\rajnishk\AppData\Local\Temp\ipykernel_15060\3685044897.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_umich[col] = df_umich[col].str.strip()


In [9]:
# df_umich['hgb'] = df_umich['hgb'].str.strip()
# df_umich.hgb[5]

In [10]:
# df_umich.dtypes

In [11]:
# df_umich.columns

In [12]:
cols_to_convert = [
    'wbc_count', 'hgb', 'hct', 'plt', 'rbc_count', 'mcv', 'mch', 
    'mchc', 'rdw', 'mpv', 'absolute_neutrophil', 'absolute_lymphocyte',
    'absolute_monocyte', 'crp'
]
for col in cols_to_convert:
    df_umich[col] = pd.to_numeric(df_umich[col], errors='coerce')     # Convert all the string numbers to numeric values for ML

C:\Users\rajnishk\AppData\Local\Temp\ipykernel_15060\2396419904.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_umich[col] = pd.to_numeric(df_umich[col], errors='coerce')     # Convert all the string numbers to numeric values for ML


In [13]:
# df_umich.hgb[5]

In [14]:
df_umich_dpi_0 = df_umich[df_umich.dpi == 0]  # Day 0 of relevant dataset from Umich. # This is a combination of training and validation.
df_JH_dpi_0 = df_JH[df_JH.dpi==0] # Day 0 of relevant dataset from JH  # This is going to be our test dataset
print("no of measurements in df_umich_dpi_0 is %s" %np.shape(df_umich_dpi_0)[0])
print("no of measurements in df_JH_dpi_0 is %s" %np.shape(df_JH_dpi_0)[0])

no of measurements in df_umich_dpi_0 is 79
no of measurements in df_JH_dpi_0 is 33


In [15]:
# df_umich_dpi_0.isna().sum()

Find out number of missing values. I will come to it later. 

In [16]:
df_umich_dpi_0.dropna(subset=['crp','ferritin','absolute_neutrophil','absolute_lymphocyte','absolute_monocyte'], inplace=True)
df_umich_dpi_0

C:\Users\rajnishk\AppData\Local\Temp\ipykernel_15060\796743494.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_umich_dpi_0.dropna(subset=['crp','ferritin','absolute_neutrophil','absolute_lymphocyte','absolute_monocyte'], inplace=True)


,studyid,dpi,wbc_count,hgb,hct,plt,rbc_count,mcv,mch,mchc,...,mpv,absolute_neutrophil,absolute_lymphocyte,absolute_monocyte,crp,ferritin,clinician_defined_crs_grade,NT_incidence,Age,Sex
5,LTFU2,0,2.8,9.3,28.6,247.0,3.01,95.0,30.9,32.5,...,9.5,2.1,0.1,0.4,0.6,1273.6,0.0,0,73,Male
20,LTFU3,0,2.8,12.7,38.1,204.0,4.21,90.5,30.2,33.3,...,9.0,2.6,0.1,0.0,4.2,211.9,0.0,0,50,Male
74,LTFU5,0,3.7,12.8,37.8,154.0,4.38,86.3,29.2,33.9,...,9.8,2.7,0.2,0.3,0.8,126.4,0.0,1,42,Male
92,LTFU6,0,0.9,8.5,26.8,127.0,2.81,95.4,30.2,31.7,...,10.9,0.7,0.0,0.0,2.5,1282.7,0.0,1,63,Male
139,LTFU8,0,2.4,14.7,44.0,103.0,4.98,88.4,29.5,33.4,...,11.3,1.7,0.3,0.3,0.5,290.2,0.0,0,64,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1523,SC73,0,2.4,11.3,34.1,146.0,3.66,93.2,30.9,33.1,...,9.7,0.9,0.0,0.1,2.8,157.0,2.0,0,63,Female
1536,SC74,0,4.1,12.0,35.9,307.0,3.54,101.4,33.9,33.4,...,9.6,3.5,0.1,0.2,1.3,195.7,0.0,1,76,Male
1556,SC75,0,1.4,9.8,27.7,152.0,3.00,92.3,32.7,35.4,...,9.2,1.3,0.0,0.0,1.2,1200.0,0.0,1,60,Male
1573,SC76,0,3.5,14.2,42.0,147.0,4.70,89.4,30.2,33.8,...,8.9,3.1,0.0,0.1,0.5,147.5,0.0,0,62,Male



dtype: objectct')


There are a few missing values. Let me fill them with the mean value for each columns for starter. I might end up doing analysis where I would just drop them. 

In [17]:
# for column in df_umich_dpi_0.select_dtypes(include=['float64', 'int64']).columns:
#     df_umich_dpi_0[column].fillna(df_umich_dpi_0[column].mean(), inplace=True)

In [18]:
# df_umich_dpi_0.isna().sum()

In [19]:
# df_JH_dpi_0.dropna(subset=['crp','ferritin'], inplace=True)
# df_JH_dpi_0.isna().sum()

Useful link for code on implementation of RFE using random forest
https://stackoverflow.com/questions/24123498/recursive-feature-elimination-on-random-forest-using-scikit-learn
Following example has been copied from this link. 

In [2]:
from sklearn import datasets
import pandas as pd
from pandas import Series
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV

class RandomForestClassifierWithCoef(RandomForestClassifier):
    def fit(self, *args, **kwargs):
        super(RandomForestClassifierWithCoef, self).fit(*args, **kwargs)
        self.coef_ = self.feature_importances_

iris = datasets.load_iris()
x=pd.DataFrame(iris.data, columns=['var1','var2','var3', 'var4'])
y=(pd.Series(iris.target, name='target')==2).astype(int)
rf = RandomForestClassifierWithCoef(n_estimators=500, min_samples_leaf=5, n_jobs=-1)
rfecv = RFECV(estimator=rf, step=1, cv=2, scoring='roc_auc', verbose=2)
selector=rfecv.fit(x, y)

C:\Users\rajnishk\AppData\Local\Temp\ipykernel_23772\2214112821.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 4 features.


In [9]:
selector.support_

array([ True, False,  True,  True])

In [1]:
# Import necessary libraries
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the Iris dataset
data = load_iris()
X = data.data
y = data.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model on the training data
rfc.fit(X_train, y_train)

# Predict on the test data
y_pred = rfc.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Detailed classification report
report = classification_report(y_test, y_pred, target_names=data.target_names)
print("Classification Report:\n", report)

Accuracy: 100.00%
Classification Report:
               precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        19
  versicolor       1.00      1.00      1.00        13
   virginica       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

